In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import tqdm

In [2]:
class Bottleneck(nn.Module):

    def __init__(self, in_channels, reduced_channels, out_channels, stride=1, starting=False):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, reduced_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(reduced_channels),
            nn.ReLU(inplace=True)
        ) 
        self.conv2 = nn.Sequential(
            nn.Conv2d(reduced_channels, reduced_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(reduced_channels),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(reduced_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
        )
        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != 4*reduced_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, 4*reduced_channels,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(4*reduced_channels)
            )

        self.final_relu = nn.ReLU(inplace=True)


    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out) + self.shortcut(x)
        out = self.final_relu(out)
        return out


In [3]:
class ResNet50(nn.Module):

    def __init__(self, num_classes=1000):
        super(ResNet50, self).__init__()
        self.num_blocks = [3,4,6,3]
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=4),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.conv2 = self._make_layer(in_channels=64, reduced_channels=64, out_channels=256, num_blocks=self.num_blocks[0], stride=1)
        self.conv3 = self._make_layer(in_channels=256, reduced_channels=128, out_channels=512, num_blocks=self.num_blocks[1], stride=1)
        self.conv4 = self._make_layer(in_channels=512, reduced_channels=256, out_channels=1024, num_blocks=self.num_blocks[2], stride=1)
        self.conv5 = self._make_layer(in_channels=1024, reduced_channels=512, out_channels=2048, num_blocks=self.num_blocks[3], stride=1)
        
        self.avg = nn.AvgPool2d(kernel_size=4)
        # FC layer, after applied 'avg pooling'
        # self.fc1 = nn.Sequential(
        #     nn.Linear(in_features=2048, out_features=1000)
        # )

        # reduce the number of channel to 17 -> number of keypoints
        self.final = nn.Sequential(
            nn.Conv2d(in_channels=2048, out_channels=17, kernel_size=1)
        )

    def _make_layer(self, in_channels, reduced_channels, out_channels, num_blocks, stride=1):
        layers = []
        strides = [stride] + [1]*(num_blocks-1)

        for i, s in enumerate(strides):
            bottleneck_layer = Bottleneck(in_channels, reduced_channels, out_channels, s)
            layers.append(bottleneck_layer)
            in_channels = reduced_channels *4
        
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)   
        out = self.conv2(out)   # output: 256*64*48
        out = self.conv3(out)   # output: 512*32*24
        out = self.conv4(out)
        out = self.conv5(out)   # output: 16*12
        #out = self.avg(out)
        #out = out.view(out.size(0), -1)

        ## the input of last layer shoul be in size 64x48 before
        out = self.final(out)
        return out

In [4]:
from torchsummary import summary

resnet = ResNet50().cuda()
summary(resnet, input_size=(3,256,192))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 64, 129, 97]           9,472
       BatchNorm2d-2          [-1, 64, 129, 97]             128
              ReLU-3          [-1, 64, 129, 97]               0
         MaxPool2d-4           [-1, 64, 64, 48]               0
            Conv2d-5           [-1, 64, 64, 48]           4,096
       BatchNorm2d-6           [-1, 64, 64, 48]             128
              ReLU-7           [-1, 64, 64, 48]               0
            Conv2d-8           [-1, 64, 64, 48]          36,864
       BatchNorm2d-9           [-1, 64, 64, 48]             128
             ReLU-10           [-1, 64, 64, 48]               0
           Conv2d-11          [-1, 256, 64, 48]          16,384
      BatchNorm2d-12          [-1, 256, 64, 48]             512
           Conv2d-13          [-1, 256, 64, 48]          16,384
      BatchNorm2d-14          [-1, 256,

In [5]:
torch.cuda.current_device()

0

In [10]:
import numpy as np
import random

def reset_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [7]:
def train(model, n_epoch, loader, optimizer, criterion, device="cpu"):
    model.train()
    for epoch in tqdm(range(n_epoch)):
        running_loss = 0.0
        for i, data in enumerate(loader, 0):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(input=outputs, target=labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print('Epoch {}, loss = {:.3f}'.format(epoch, running_loss/len(loader)))
    
    print('Training Finished')

In [8]:
def test(model, loader, device="cpu"):
    model.eval()
    total=0
    correct=0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted==labels).sum().item()
    acc = 100*correct/total
    
    return acc


In [13]:
reset_seed(0)
resnet_model = ResNet50().to("cuda")
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=resnet_model.parameters(), lr=0.1, momentum=0.9)

## train
train(model=resnet_model, n_epoch=10, loader=trainloader, optimizer=optimizer, criterion=criterion, device="cuda")
## test
resnet_acc = test(resnet_model, testloader, device="cuda")

print('ResNet Test accuracy: {:.2f}%'.format(resnet_acc))

NameError: name 'trainloader' is not defined